In [ ]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v6 - TIMING FIX
El problema era que no esperábamos suficiente las respuestas
"""

import hashlib
import struct
import socket
import time
import random
import select
from typing import List, Tuple, Optional, Dict
from dataclasses import dataclass, field
from collections import defaultdict
from datetime import datetime

@dataclass
class BlockHeader:
    version: int
    prev_hash: bytes
    merkle_root: bytes
    timestamp: int
    bits: int
    nonce: int

    def serialize(self) -> bytes:
        return (struct.pack('<I', self.version) + self.prev_hash +
                self.merkle_root + struct.pack('<I', self.timestamp) +
                struct.pack('<I', self.bits) + struct.pack('<I', self.nonce))

    def hash(self) -> bytes:
        return hashlib.sha256(hashlib.sha256(self.serialize()).digest()).digest()

    def hash_hex(self) -> str:
        return self.hash()[::-1].hex()


@dataclass
class Block:
    header: BlockHeader
    coinbase_script: bytes = b''

    def serialize(self) -> bytes:
        # Header
        data = self.header.serialize()
        # 1 transaction
        data += b'\x01'
        # Coinbase tx
        data += self.create_coinbase()
        return data

    def create_coinbase(self) -> bytes:
        tx = struct.pack('<I', 1)  # version
        tx += b'\x01'  # input count
        tx += bytes(32)  # prev txid
        tx += struct.pack('<I', 0xffffffff)  # prev vout
        script = self.coinbase_script or b'\x04KAORU'
        tx += bytes([len(script)]) + script
        tx += struct.pack('<I', 0xffffffff)  # sequence
        tx += b'\x01'  # output count
        tx += struct.pack('<Q', 50 * 10**8)  # 50 BTC
        out_script = b'\x76\xa9\x14' + bytes(20) + b'\x88\xac'
        tx += bytes([len(out_script)]) + out_script
        tx += struct.pack('<I', 0)  # locktime
        return tx


class KaoruBridgeV51:
    """v5.1 - Fixed timing issues"""

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')

    DNS_SEEDS = [
        'seed.bitcoin.sipa.be',
        'dnsseed.bluematt.me',
        'seed.bitcoinstats.com',
    ]

    MSG_BLOCK = 2

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.our_blocks: List[Block] = []
        self.stats = defaultdict(int)

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    def create_block(self, height: int, prev_hash: bytes, message: str) -> Block:
        script = bytes([0x01, height]) + message.encode()[:50]
        merkle = self.double_sha256(script)

        header = BlockHeader(
            version=0x20000000,
            prev_hash=prev_hash,
            merkle_root=merkle,
            timestamp=int(time.time()),
            bits=0x1d00ffff,
            nonce=0
        )
        return Block(header=header, coinbase_script=script)

    def mine_block(self, block: Block, target_zeros: int = 4) -> bool:
        target = "0" * target_zeros
        for nonce in range(50_000_000):
            block.header.nonce = nonce
            if block.header.hash_hex().startswith(target):
                return True
        return False

    def build_chain(self, num_blocks: int = 3, target_zeros: int = 4) -> List[Block]:
        print(f"\n   🔗 Construyendo {num_blocks} bloques...")
        prev_hash = bytes(32)

        for i in range(num_blocks):
            block = self.create_block(i, prev_hash, f"KAORU_BLOCK_{i}")
            if self.mine_block(block, target_zeros):
                self.our_blocks.append(block)
                prev_hash = block.header.hash()
                print(f"      [{i}] {block.header.hash_hex()[:40]}... ✅")
            else:
                print(f"      [{i}] ❌ Falló")
                break

        return self.our_blocks

    # ============================================
    # MENSAJES
    # ============================================

    def create_message(self, command: str, payload: bytes) -> bytes:
        cmd = command.encode().ljust(12, b'\x00')
        length = struct.pack('<I', len(payload))
        checksum = self.double_sha256(payload)[:4]
        return self.magic + cmd + length + checksum + payload

    def create_version(self, ip: str) -> bytes:
        payload = struct.pack('<i', 70015)  # version
        payload += struct.pack('<Q', 1)  # services
        payload += struct.pack('<q', int(time.time()))  # timestamp
        payload += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + socket.inet_aton(ip) + struct.pack('>H', 8333)
        payload += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + b'\x00\x00\x00\x00' + struct.pack('>H', 8333)
        payload += struct.pack('<Q', random.randint(0, 2**64-1))  # nonce
        payload += b'\x12/KaoruBridge:5.1.0/'  # user agent
        payload += struct.pack('<i', len(self.our_blocks))  # height
        payload += b'\x01'  # relay
        return self.create_message('version', payload)

    def create_inv(self) -> bytes:
        payload = bytes([len(self.our_blocks)])
        for block in self.our_blocks:
            payload += struct.pack('<I', self.MSG_BLOCK)
            payload += block.header.hash()
        return self.create_message('inv', payload)

    def create_headers(self) -> bytes:
        payload = bytes([len(self.our_blocks)])
        for block in self.our_blocks:
            payload += block.header.serialize()
            payload += b'\x00'
        return self.create_message('headers', payload)

    def parse_messages(self, data: bytes) -> List[Tuple[str, bytes]]:
        msgs = []
        i = 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except:
                break
        return msgs

    # ============================================
    # CONEXIÓN MEJORADA
    # ============================================

    def recv_all(self, sock: socket.socket, timeout: float = 3.0) -> bytes:
        """Recibe todos los datos disponibles."""
        sock.setblocking(False)
        data = b''
        end_time = time.time() + timeout

        while time.time() < end_time:
            try:
                ready = select.select([sock], [], [], 0.1)
                if ready[0]:
                    chunk = sock.recv(8192)
                    if chunk:
                        data += chunk
                    else:
                        break
            except:
                break

        sock.setblocking(True)
        return data

    def handle_peer(self, ip: str, port: int = 8333) -> Dict:
        result = {
            'ip': ip,
            'connected': False,
            'handshake': False,
            'messages_recv': [],
            'messages_sent': [],
            'headers_requested': False,
            'blocks_sent': 0,
            'peer_agent': ''
        }

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(10)
            sock.connect((ip, port))
            result['connected'] = True

            # === HANDSHAKE ===
            sock.send(self.create_version(ip))
            result['messages_sent'].append('version')
            self.stats['sent_version'] += 1

            # Esperar VERSION del peer
            time.sleep(0.5)
            data = self.recv_all(sock, 2.0)

            for cmd, payload in self.parse_messages(data):
                result['messages_recv'].append(cmd)
                self.stats[f'recv_{cmd}'] += 1

                if cmd == 'version':
                    # Extraer user agent
                    try:
                        offset = 80
                        ua_len = payload[offset]
                        result['peer_agent'] = payload[offset+1:offset+1+ua_len].decode('ascii', errors='ignore')
                    except:
                        pass

                    # Enviar VERACK
                    sock.send(self.create_message('verack', b''))
                    result['messages_sent'].append('verack')
                    self.stats['sent_verack'] += 1
                    result['handshake'] = True

            if not result['handshake']:
                sock.close()
                return result

            # === ANUNCIAR BLOQUES ===
            time.sleep(0.3)
            sock.send(self.create_inv())
            result['messages_sent'].append('inv')
            self.stats['sent_inv'] += 1

            # === LOOP DE COMUNICACIÓN ===
            for round_num in range(5):
                time.sleep(0.5)
                data = self.recv_all(sock, 2.0)

                if not data:
                    continue

                for cmd, payload in self.parse_messages(data):
                    result['messages_recv'].append(cmd)
                    self.stats[f'recv_{cmd}'] += 1

                    if cmd == 'ping':
                        sock.send(self.create_message('pong', payload))
                        result['messages_sent'].append('pong')
                        self.stats['sent_pong'] += 1

                    elif cmd == 'sendcmpct':
                        sock.send(self.create_message('sendcmpct', b'\x00' + struct.pack('<Q', 1)))
                        result['messages_sent'].append('sendcmpct')
                        self.stats['sent_sendcmpct'] += 1

                    elif cmd == 'getheaders':
                        result['headers_requested'] = True
                        print(f"      🎯 GETHEADERS! Enviando {len(self.our_blocks)} headers...")
                        sock.send(self.create_headers())
                        result['messages_sent'].append('headers')
                        self.stats['sent_headers'] += 1

                    elif cmd == 'getdata':
                        count = payload[0] if payload else 0
                        print(f"      🎯 GETDATA! Solicitan {count} items...")

                        offset = 1
                        for _ in range(count):
                            if offset + 36 > len(payload):
                                break
                            inv_type = struct.unpack('<I', payload[offset:offset+4])[0]
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36

                            if inv_type == self.MSG_BLOCK:
                                for block in self.our_blocks:
                                    if block.header.hash() == inv_hash:
                                        print(f"      📦 Enviando bloque {block.header.hash_hex()[:20]}...")
                                        sock.send(self.create_message('block', block.serialize()))
                                        result['messages_sent'].append('block')
                                        result['blocks_sent'] += 1
                                        self.stats['sent_block'] += 1

                    elif cmd == 'verack':
                        pass  # Ya manejado

                    elif cmd == 'reject':
                        print(f"      ❌ REJECT: {payload[:30]}")

                    elif cmd == 'inv':
                        count = payload[0] if payload else 0
                        print(f"      📥 INV: Nodo ofrece {count} items")

            sock.close()

        except Exception as e:
            result['error'] = str(e)

        return result

    def discover_nodes(self) -> List[Tuple[str, int]]:
        nodes = []
        for seed in self.DNS_SEEDS:
            try:
                ips = socket.gethostbyname_ex(seed)[2]
                nodes.extend((ip, 8333) for ip in ips[:8])
            except:
                pass
        random.shuffle(nodes)
        return nodes

    # ============================================
    # EJECUCIÓN
    # ============================================

    def execute(self, num_blocks: int = 3, target_zeros: int = 4, num_peers: int = 8):
        print("""
╔════════════════════════════════════════════════════════════════════════════════╗
║                                                                                ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗███████╗   ██╗       ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║██╔════╝  ███║       ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║███████╗  ╚██║       ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝╚════██║   ██║       ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝ ███████║██╗██║       ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝  ╚══════╝╚═╝╚═╝       ║
║                                                                                ║
║                            TIMING FIX EDITION- era 6 XD                        ║
║                                                                                ║
╚════════════════════════════════════════════════════════════════════════════════╝
        """)

        # FASE 1
        print("=" * 80)
        print("[FASE 1] ⛏️  CONSTRUYENDO BLOCKCHAIN")
        print("=" * 80)

        self.build_chain(num_blocks, target_zeros)
        print(f"\n   Total: {len(self.our_blocks)} bloques minados")

        # FASE 2
        print("\n" + "=" * 80)
        print("[FASE 2] 🌐 DESCUBRIENDO NODOS")
        print("=" * 80)

        nodes = self.discover_nodes()
        print(f"\n   Encontrados: {len(nodes)} nodos")

        # FASE 3
        print("\n" + "=" * 80)
        print("[FASE 3] 📡 PROPAGANDO A LA RED")
        print("=" * 80)

        results = []

        for ip, port in nodes[:num_peers]:
            print(f"\n   ┌─ 📡 {ip}")

            result = self.handle_peer(ip, port)
            results.append(result)

            if result['connected']:
                print(f"   │  ✅ Conectado")
                if result['peer_agent']:
                    print(f"   │  👤 {result['peer_agent'][:40]}")
                print(f"   │  📥 Recv: {', '.join(result['messages_recv'][:6])}")
                print(f"   │  📤 Sent: {', '.join(result['messages_sent'][:6])}")
                print(f"   │  🎯 Headers pedidos: {'✅' if result['headers_requested'] else '❌'}")
                print(f"   │  📦 Bloques enviados: {result['blocks_sent']}")
                print(f"   └─ ✅")
            else:
                error = result.get('error', 'Unknown')[:30]
                print(f"   └─ ❌ {error}")

        # RESULTADOS
        print("\n" + "=" * 80)
        print("                         📊 RESULTADOS FINALES")
        print("=" * 80)

        connected = sum(1 for r in results if r['connected'])
        handshakes = sum(1 for r in results if r['handshake'])
        headers_req = sum(1 for r in results if r['headers_requested'])
        blocks_sent = sum(r['blocks_sent'] for r in results)

        print(f"""
   ╔═══════════════════════════════════════════════════════════════════════╗
   ║                         ESTADÍSTICAS                                  ║
   ╠═══════════════════════════════════════════════════════════════════════╣
   ║                                                                       ║
   ║   Bloques minados:           {len(self.our_blocks):3d}                                   ║
   ║   Peers contactados:         {len(results):3d}                                   ║
   ║   Conexiones exitosas:       {connected:3d}                                   ║
   ║   Handshakes completos:      {handshakes:3d}                                   ║
   ║   Headers solicitados:       {headers_req:3d}                                   ║
   ║   Bloques enviados:          {blocks_sent:3d}                                   ║
   ║                                                                       ║
   ╚═══════════════════════════════════════════════════════════════════════╝
        """)

        print("   📊 Mensajes por tipo:")
        for key in sorted(self.stats.keys()):
            direction = "📤" if key.startswith('sent_') else "📥"
            cmd = key.split('_', 1)[1]
            print(f"      {direction} {cmd:15s}: {self.stats[key]}")

        print(f"""
   ╔═══════════════════════════════════════════════════════════════════════╗
   ║                                                                       ║
   ║   🏆 RESUMEN:                                                         ║
   ║                                                                       ║
   ║   {'✅' if len(self.our_blocks) > 0 else '❌'} Blockchain construida con PoW                                 ║
   ║   {'✅' if connected > 0 else '❌'} Conexiones a mainnet                                        ║
   ║   {'✅' if handshakes > 0 else '❌'} Handshakes VERSION/VERACK                                  ║
   ║   {'✅' if headers_req > 0 else '❌'} Nodos pidieron nuestros HEADERS                           ║
   ║   {'✅' if blocks_sent > 0 else '❌'} Bloques completos enviados                                ║
   ║                                                                       ║
   ╚═══════════════════════════════════════════════════════════════════════╝

   Estado: KAORU BRIDGE v5.1 COMPLETADO ✅
        """)

        return results


# ============================================
# EJECUCIÓN
# ============================================

if __name__ == "__main__":
    bridge = KaoruBridgeV51()
    bridge.execute(num_blocks=3, target_zeros=4, num_peers=10)


╔════════════════════════════════════════════════════════════════════════════════╗
║                                                                                ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗███████╗   ██╗       ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║██╔════╝  ███║       ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║███████╗  ╚██║       ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝╚════██║   ██║       ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝ ███████║██╗██║       ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝  ╚══════╝╚═╝╚═╝       ║
║                                                                                ║
║                            TIMING FIX EDITION- era 6 XD                        ║
║                                                                                ║
╚════════════════════════════════════════════════════════════════════════════════╝
   